<a href="https://colab.research.google.com/github/truongthuanr/self-project/blob/dev/11_Named_Entity_Recognition/Named_Entity_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

%%capture
!pip3 install datasets


In [2]:

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from datasets import load_dataset
import os


In [3]:
from keras.models import Model
from keras.layers import Input,Embedding,TimeDistributed,\
                         Dropout,Conv1D,MaxPooling1D,\
                         Flatten,Bidirectional,LSTM,Dense,\
                         concatenate

from keras.initializers import RandomUniform

from keras.optimizers import Adam





In [4]:
# For vetorization
max_word_tokens = 24000
max_sentence_length = 30


# Dataset

Used dataset:

https://huggingface.co/datasets/conll2003

In [5]:
conll_data = load_dataset("conll2003")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
type(conll_data['train']['tokens'][0])

list

In [7]:
conll_data['train']['ner_tags'][0]

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [8]:
from keras.preprocessing.text import Tokenizer

In [9]:
# tokenizer = Tokenizer(num_words=max_word_tokens)
# tokenizer.fit_on_texts(conll_data['train']['tokens'])

## Embedding

In [12]:
# '''
# def export_to_file(export_file_path, data):
#     with open(export_file_path, "w") as f:
#         for record in data:
#             ner_tags = record["ner_tags"]
#             tokens = record["tokens"]
#             if len(tokens) > 0:
#                 f.write(
#                     str(len(tokens))
#                     + "\t"
#                     + "\t".join(tokens)
#                     + "\t"
#                     + "\t".join(map(str, ner_tags))
#                     + "\n"
#                 )


# os.mkdir("data")
# export_to_file("./data/conll_train.txt", conll_data["train"])
# export_to_file("./data/conll_val.txt", conll_data["validation"])
# '''

# Data preprocessing


In [57]:
# X_train = np.array([ " ".join(row) for row in conll_data['train']['tokens']])
X_train = conll_data['train']['tokens']

In [58]:
X_train[0]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [ ]:
# y_train = np.array([np.array(y) for y in conll_data['train']['ner_tags']])
# y_train = pd.Series(conll_data['train']['ner_tags'])
# y_train = tf.convert_to_tensor(conll_data['train']['ner_tags'])

In [51]:
type(y_train[0])

list

# Model

In [15]:
from keras.layers import TextVectorization

In [16]:
# Configuration



In [77]:
text_vectorizer = TextVectorization(max_tokens=max_word_tokens,   # how many word in the vocab lib
                                    standardize='lower_and_strip_punctuation', # process the tab
                                    split='whitespace', # split the token
                                    ngrams=None,
                                    output_mode='int', # map token to number
                                    output_sequence_length=max_sentence_length,
                                    pad_to_max_tokens=False,
                                    vocabulary=None,)
text_vectorizer.adapt(np.array([ " ".join(row) for row in conll_data['train']['tokens']]))

AttributeError: The layer "text_vectorization_4" has never been called and thus has no defined output shape.

In [19]:
print(text_vectorizer.vocabulary_size())
print(text_vectorizer.get_vocabulary()[:10])

20070
['', '[UNK]', 'the', 'of', 'in', 'to', 'a', 'and', 'on', 'said']


In [20]:
char2Idx = {"PADDING":0, "UNKNOWN":1}
for c in " 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,-_()[]{}!?:;#'\"/\\%$`&=*+@^~|":
    char2Idx[c] = len(char2Idx)


len(char2Idx)

95

In [21]:
## Embeding




In [78]:
# Word level
word_input = Input(shape=(1,),dtype="string",name='word_input')
# Tokenize
words = text_vectorizer(word_input)

# Embedding
embedding = Embedding(input_dim=max_word_tokens,
                      output_dim=128,
                      embeddings_initializer="uniform",
                      input_length=max_sentence_length)
words = embedding(word_input)


ValueError: Exception encountered when calling layer "text_vectorization_5" (type TextVectorization).

When using `TextVectorization` to tokenize strings, the input rank must be 1 or the last shape dimension must be 1. Received: inputs.shape=(None, None) with rank=2

Call arguments received by layer "text_vectorization_5" (type TextVectorization):
  • inputs=tf.Tensor(shape=(None, None), dtype=string)

In [55]:
# Character level
character_input=Input(shape=(None,52,),
                      name='char_input')
embed_char_out=TimeDistributed(Embedding(input_dim = len(char2Idx),
                                         output_dim=30,
                                         embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)),
                                         name='char_embedding')(character_input)
dropout = Dropout(0.5)(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3,
                                   filters=30,
                                   padding='same',activation='tanh', strides=1))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(max_sentence_length))(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)

In [46]:
char

<KerasTensor: shape=(None, None, 30) dtype=float32 (created by layer 'dropout_7')>

In [51]:

combined = concatenate([words, char])
# combined = words
# Bi-LSTM
output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(combined)

# # Output -> Tag Scoring -> Tag
output = TimeDistributed(Dense(max_sentence_length, activation='softmax'))(output)

model = Model(inputs=[word_input, character_input], outputs=[output])


<KerasTensor: shape=(None, 30, 400) dtype=float32 (created by layer 'bidirectional_8')>

In [52]:
model.compile(loss="binary_crossentropy",
              optimizer=Adam(),
              metrics=["accuracy"])

In [53]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 char_input (InputLayer)     [(None, None, 52)]           0         []                            
                                                                                                  
 char_embedding (TimeDistri  (None, None, 52, 30)         2850      ['char_input[0][0]']          
 buted)                                                                                           
                                                                                                  
 dropout_6 (Dropout)         (None, None, 52, 30)         0         ['char_embedding[0][0]']      
                                                                                                  
 time_distributed_9 (TimeDi  (None, None, 52, 30)         2730      ['dropout_6[0][0]']       

In [ ]:
# Charater level

# Character Embedding

# Additional character feature

# Convolution

# Max

# CNN extracted character feature


In [ ]:
# # Word level
# words_input = Input(shape=(None,),dtype='int32',name='words_input')
# words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)

# # Casing
# casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
# casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)

# # Character level
# character_input=Input(shape=(None,52,),name='char_input')
# embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
# dropout= Dropout(0.5)(embed_char_out)
# conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1))(dropout)
# maxpool_out=TimeDistributed(MaxPooling1D(52))(conv1d_out)
# char = TimeDistributed(Flatten())(maxpool_out)
# char = Dropout(0.5)(char)



# output = concatenate([words, casing,char])
# output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(output)

# output = TimeDistributed(Dense(len(label2Idx), activation='softmax'))(output)

# model = Model(inputs=[words_input, casing_input,character_input], outputs=[output])
# model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam')
# model.summary()


In [31]:
output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(combined)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [52]:
model.fit(X_train[:1000],
          y_train[:1000],
          epochs=5,
          validation_data=(X_train[1000:1100], y_train[1000:1100]))

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

# References

https://keras.io/examples/nlp/ner_transformers/